<a href="https://colab.research.google.com/github/kabeerbora/1980s_delin/blob/main/delin_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing Packages

In [1]:
import pandas as pd
!pip install pyreadstat
import pandas as pd
import pyreadstat
import os
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.1/620.1 kB 8.2 MB/s eta 0:00:00
Mounted at /content/drive


#Extracting columns

In [ ]:
import os
import pyreadstat
import pandas as pd

# Define the base directory
base_directory = '/content/drive/MyDrive/data_economics_research/ASI_Firms'

# Specify the folders to process (from 1976_1977 to 1989_1990)
folders_to_process = [f"{year}_{year+1}" for year in range(1976, 1991)]

# Initialize a list to store the year and column labels
columns_data = []

# Loop through all the specified folders and process the .sav files
for folder in folders_to_process:
    folder_path = os.path.join(base_directory, folder)

    # Check if the folder exists before processing
    if os.path.exists(folder_path):
        # List all .sav files in the current folder
        sav_files = [f for f in os.listdir(folder_path) if f.endswith('.sav')]

        # Loop through all the .sav files and get column labels
        for file in sav_files:
            file_path = os.path.join(folder_path, file)

            # Read the .sav file
            df, meta = pyreadstat.read_sav(file_path)

            # Get the column labels and add each label as a separate row
            for col_label in meta.column_labels:
                columns_data.append({
                    'Year': folder,
                    'File': file,
                    'Column Label': col_label
                })
    else:
        print(f"Folder {folder} does not exist. Skipping...")

# Create a DataFrame from the collected data
columns_df = pd.DataFrame(columns_data)

# Display the resulting DataFrame
columns_df.head()
output_path = '/content/drive/MyDrive/data_economics_research/ASI_Firms/columns_1976_1989.xlsx'
# columns_df.to_excel(output_path, index=False)

#Extracting Data

In [4]:
import pandas as pd
import os

alias_file_path = "/content/drive/MyDrive/data_economics_research/ASI_Firms/columns_1976_1989.xlsx"
alias_df = pd.read_excel(alias_file_path)


base_path = "/content/drive/MyDrive/data_economics_research/ASI_Firms"
years = alias_df["Year"].unique()
available_years = []

for year in years:
    year_str = str(year)
    folder_path = os.path.join(base_path, year_str)
    if os.path.exists(folder_path):
        if any(f.endswith(".sav") and (f.startswith("ASI") or f.startswith("ASISUM")) for f in os.listdir(folder_path)):
            available_years.append(year)

available_years

['1976_1977',
 '1977_1978',
 '1979_1980',
 '1980_1981',
 '1981_1982',
 '1982_1983',
 '1983_1984',
 '1984_1985',
 '1985_1986',
 '1986_1987',
 '1987_1988',
 '1988_1989',
 '1990_1991']

In [6]:
import pandas as pd
import os
import pyreadstat


alias_file_path = "/content/drive/MyDrive/data_economics_research/ASI_Firms/columns_1976_1989.xlsx"
alias_df = pd.read_excel(alias_file_path)

alias_df = alias_df.dropna(subset=['alias'])
alias_df['Column Label'] = alias_df['Column Label'].str.strip()
alias_df['alias'] = alias_df['alias'].str.strip()

label_to_alias = dict(zip(alias_df['Column Label'], alias_df['alias']))
valid_labels = set(label_to_alias.keys())

dfs = []
for folder_name in available_years:
    folder_path = f"/content/drive/MyDrive/data_economics_research/ASI_Firms/{folder_name}"
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith(".sav") and (filename.startswith("ASI") or filename.startswith("ASISUM")):
                file_path = os.path.join(folder_path, filename)

                df, meta = pyreadstat.read_sav(file_path, apply_value_formats=True)

                column_labels = meta.column_names_to_labels
                df.columns = [column_labels.get(col, col) for col in df.columns]

                df = df.loc[:, df.columns.isin(valid_labels)]

                df.rename(columns=label_to_alias, inplace=True)

                df['year'] = folder_name

                dfs.append(df)
df_final = pd.concat(dfs, ignore_index=True)
# df_final.to_csv('/content/drive/MyDrive/data_economics_research/jan_weber_kabeer_bora/1976_1988_allfirms.csv', index=False)

,state_code,rsl,nic_code,ownership_code,organization_code,scheme_code,district_code,year_initial,capital_open,capital_closing,...,wages,tot_output,value_added,semi_open,semi_close,bonus_workers,tot_emoluments,K_formation,year,gross_sales
0,RAJASTHAN,7.0,2010.0,Wholly State and/or Local Government,Partnership : Others,Census,10.0,1972.0,0.0,12495400.0,...,0.0,48890500.0,7632500.0,0.0,0.0,0.0,727100.0,0.0,1976_1977,NaN
1,RAJASTHAN,10.0,2010.0,Joint Sector Private,Public Corporations,Census,10.0,1972.0,0.0,4937300.0,...,0.0,17799200.0,2359200.0,0.0,0.0,0.0,483900.0,0.0,1976_1977,NaN
2,RAJASTHAN,90035.0,2041.0,NR,NR,Sample,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1976_1977,NaN
3,RAJASTHAN,90034.0,2041.0,NR,NR,Sample,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1976_1977,NaN
4,RAJASTHAN,90055.0,2041.0,NR,NR,Sample,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1976_1977,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712364,DAMAN & DIU,28607.0,3751.0,Joint Sector Public,Private Limited Company,100 or more workers,1.0,1979.0,925763.0,826652.0,...,934150.0,18277826.0,4878716.0,156982.0,60019.0,144262.0,1266495.0,169830.0,1990_1991,19117132.0
712365,DAMAN & DIU,39.0,3751.0,Joint Sector Public,Public Limited Company,Complete Enumeration,1.0,1979.0,561620.0,501165.0,...,892574.0,13564541.0,1878302.0,38313.0,100662.0,130453.0,1221991.0,88576.0,1990_1991,15808346.0
712366,DAMAN & DIU,40.0,3758.0,Joint Sector Public,Public Limited Company,Complete Enumeration,1.0,1979.0,431476.0,545225.0,...,655312.0,25713388.0,4637501.0,242787.0,133400.0,93250.0,1132255.0,-247144.0,1990_1991,30963184.0
712367,DAMAN & DIU,28609.0,3758.0,Joint Sector Public,Public Limited Company,100 or more workers,1.0,100.0,561620.0,501165.0,...,892574.0,13407259.0,1751020.0,38313.0,100662.0,130453.0,1221991.0,88576.0,1990_1991,15651064.0
